<a href="https://colab.research.google.com/github/aghadavood/persian-qa-system/blob/main/Gpt4api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Assumptions:
Purpose: Create embeddings for text data.
Model: Use "gpt-4o-mini" for creating embeddings.
Data: Your JSON file contains fields like title and abstract.

In [ ]:
!pip install --upgrade openai
!pip install ijson
!pip install langchain
!pip install langchain-openai
!pip install faiss-cpu
!pip install tiktoken
!pip install openai
!pip install senetence-transformers
!pip install langchain-community # install the missing package
!pip install sentence_transformers
!pip install --upgrade langchain langchain_community openai tqdm ijson sentence-transformers
!pip install transformers
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

In [ ]:
import os
from pathlib import Path
import hashlib
from tqdm import tqdm
import ijson
import time
import random
import pickle
import io

# Langchain imports
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# OpenAI import
from openai import OpenAI

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
json_file_path = '/content/drive/My Drive/corpus-engineering.json'

In [ ]:

OpenAI.api_key = ""
client = OpenAI(api_key=OpenAI.api_key)

# Use a local embedding model
embeddings = HuggingFaceEmbeddings(model_name="distiluse-base-multilingual-cased-v2")

# Text splitter for Persian
text_splitter = RecursiveCharacterTextSplitter(
    separators=['\n', '،', '؛', '.'],  # Persian-specific separators
    chunk_size=1000,
    chunk_overlap=50
)

# Prepare document from a single JSON item
def prepare_document(item):
    text = f"شناسه: {item['id']}\nعنوان: {item['title']}\nچکیده: {item['abstract']}\nموضوع اول: {item['FirstSubject']}\nموضوع دوم: {item['SecondSubject']}"
    return text_splitter.create_documents([text])

# Create or load vector database


  # Process documents in batches
def process_documents(file_path, batch_size=1000):
    with open(file_path, 'rb') as f:
        # Get total number of items for progress bar
        parser = ijson.parse(f)
        total_items = sum(1 for _ in parser if _ == ('','start_map'))

    with open(file_path, 'rb') as f:
        items = ijson.items(f, 'item')
        batch = []
        for item in tqdm(items, total=total_items, desc="Processing documents"):
            batch.extend(prepare_document(item))
            if len(batch) >= batch_size:
                yield batch
                batch = []
        if batch:
            yield batch

def get_vector_db(file_path, force_rebuild=False):
    # Create 'embeddings' folder in the current working directory
    embeddings_folder = Path("embeddings")
    embeddings_folder.mkdir(exist_ok=True)

    file_hash = hashlib.md5(Path(file_path).read_bytes()).hexdigest()
    faiss_index_path = embeddings_folder / f"{file_hash}.faiss"
    index_metadata_path = embeddings_folder / f"{file_hash}_metadata.pkl"

    if not force_rebuild and faiss_index_path.exists() and index_metadata_path.exists():
        print(f"Loading existing FAISS index from {faiss_index_path}...")
        vector_db = FAISS.load_local(str(faiss_index_path), embeddings , allow_dangerous_deserialization=True)
        with open(index_metadata_path, 'rb') as f:
            vector_db.docstore._dict = pickle.load(f)
        return vector_db

    print("Building new FAISS index...")
    vector_db = None
    for batch in process_documents(file_path):
        if vector_db is None:
            vector_db = FAISS.from_documents(batch, embeddings)
        else:
            vector_db.add_documents(batch)

    print(f"Saving FAISS index to {faiss_index_path}...")
    vector_db.save_local(str(faiss_index_path))
    with open(index_metadata_path, 'wb') as f:
        pickle.dump(vector_db.docstore._dict, f)

    return vector_db

def generate_answer(query, contents, max_retries=5):
    prompt = f"به زبان فارسی به پرسش زیر پاسخ دهید. پاسخ را بر اساس اطلاعات داده شده ارائه دهید:\n\nپرسش: {query}\n\nاطلاعات:\n{contents}"
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
            )
            return response.choices[0].message.content
        except Exception as e:
            if attempt == max_retries - 1:
                raise e
            time.sleep(2 ** attempt + random.random())

def main():
    vector_db = get_vector_db(json_file_path)

    while True:
        query = input("سوال خود را به فارسی بپرسید (یا 'خروج' را برای پایان وارد کنید): ")
        if query.lower() == 'خروج':
            break

        results = vector_db.similarity_search(query, k=2)
        contents = "\n".join([doc.page_content for doc in results])

        answer = generate_answer(query, contents)
        print(f"پاسخ: {answer}\n")

if __name__ == "__main__":
    main()

